# Operational Benchmarking

1. Setup and build the LLRS project.
2. Configure the benchmark by modifying the `config/operational-benchmarking/config.yml` file.
3. Run the following notebook to generate the benchmark results.

- The production-quality figure generation of this notebook uses the `Experiment` module's Python package, make sure to install it. (The guide to install the package is available in the `README.md` file of the `Experiment` module.)

In [ ]:
import os
import time
import yaml
import pickle
import numpy as np
import subprocess
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import sys
from pathlib import Path
import json
import datetime
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
project_dir = Path().resolve().parents[1]
sys.path.insert(0, project_dir)


In [ ]:
executable_path = str(project_dir) + "/bin/modules/operational-benchmarking/op-bench"

Choose the config file for the benchmark. The default value is `config/benchmark_params_cpp.yml`, the path given must be relative to the `LLRS` directory.


In [ ]:
config_path = str(project_dir) + "/configs/operational-benchmarking/config.yml"

In [ ]:
# user-defined params
with open(config_path, 'r') as file:
    benchmark_params = yaml.safe_load(file)

benchmark_names = []
benchmarks = []
for it in benchmark_params["range_param"]:
    benchmark_names.append("benchmark_" + str(it))
    benchmarks.append([benchmark_params["num_trials"], benchmark_params["num_repetition"], it])
    print(f"Creating: {it}")

variable_param = benchmark_params["variable_param"]

# create runtime directories
resources_dir = str(project_dir) + "/resources/runtime/operational_benchmarking/" + str(time.time())
Path(resources_dir).mkdir(parents=True, exist_ok=True)

# create the temp config file
with open(str(project_dir) + "/resources/problems/" + benchmark_params['base_problem_definition'], 'r') as file:
    config_str = file.read()


In [ ]:
data = []

for it in benchmarks:
    created_config = resources_dir + "/config.yml"
    with open(created_config, "w") as file:
        file.write(config_str.format(it[2]))
    result = subprocess.Popen([executable_path, created_config, str(it[0]), str(it[1])], stdout=subprocess.PIPE, text=True)
    result.wait()
    # Display the output
    raw_output = result.stdout.read().strip().replace('\r\n', '\n').replace('\r', '')
    data.append(float(raw_output))
    print("Processed C++ Output:", data[-1]) 

In [ ]:
# OPTIONAL: Save the data results  in a pickle file
filename = str(project_dir) + "/resources/pickled_results/" + time.time() + ".pkl"
data_dict = {benchmark_params['variable_param']: benchmark_params["range_param"], 'mean_success_probability': data}
with open(filename, "wb") as f:
    pickle.dump(data_dict, f)

### Generate Operational Benchmarking S-Shape Curve ###

In [ ]:
import experiment.toolkits.figure_formatting.formatter_2023_az as fmt
import experiment.toolkits.figure_formatting.production_quality_plots as prod_plot

In [ ]:
# If plotting only a single data set, which was gathered above, use this cell
x_list = benchmark_params["range_param"]
fig, ax = plt.subplots() 
ax.set_xlabel(benchmark_params["variable_param"])
ax.set_ylabel("Success Rate (p)")

if min(x_list) == max(x_list):
    ax.set_xlim(min(x_list) - 1, max(x_list)+ 1)
else:   
    ax.set_xlim(min(x_list), max(x_list))
ax.set_ylim(0,1)


prod_plot.production_quality_scatter(fig, ax, [x_list], [data], num_xticks = 3, save_path=resources_dir + "/mean_success.pdf")

In [ ]:
# If plotting multiple saved datasets which are being read from pickle files, use this cell

with open('FILENAME', 'rb') as f:
    pickle_data = pickle.load(f)
    data_1 = pickle_data['mean_success_probability']

with open('FILENAME', 'rb') as f:
    pickle_data = pickle.load(f)
    data_2 = pickle_data['mean_success_probability']

with open('FILENAME', 'rb') as f:
    pickle_data = pickle.load(f)
    data_3 = pickle_data['mean_success_probability']


fig, ax = plt.subplots() 
ax.set_xlabel(benchmark_params["variable_param"])
ax.set_ylabel("Success rate (p)")

x_list = pickle_data[benchmark_params['variable_param']]
if min(x_list) == max(x_list):
    ax.set_xlim(min(x_list) - 1, max(x_list)+ 1)
else:   
    ax.set_xlim(min(x_list), max(x_list))
ax.set_ylim(0,1)

prod_plot.production_quality_scatter(fig, ax, [x_list, x_list, x_list], [data_1, data_2, data_3], num_xticks = 4, save_path=resources_dir + "/mean_success_probability.pdf")
